In [416]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import os
import urllib
url = "https://covid19.who.int/WHO-COVID-19-global-data.csv"
file_path = os.path.join("data", "covid")
os.makedirs(file_path, exist_ok=True)
csv_path = os.path.join(file_path, "WHO-COVID-19-global-data.csv")
urllib.request.urlretrieve(url, csv_path)
df = pd.read_csv(csv_path)
df2 = pd.read_csv("data/World-GDP.csv")
df.columns = [col.strip() for col in df.columns]
df['Date_reported'] = pd.to_datetime(df["Date_reported"])
isO = df['Date_reported']==pd.to_datetime('2021-02-21 00:00:00')
df = df[isO]
df = df.reset_index()
df.drop(columns=["New_deaths", "New_cases", "index", 'WHO_region', "Country_code", "Date_reported"], inplace=True)
df.rename(columns={'Country': "Area"}, inplace=True)
df2.rename(columns={'Country or Area': "Area"}, inplace=True)
df = df[df.Cumulative_deaths > 0]
df = df[df.Cumulative_cases > 20000]
df2.drop(columns=["Year", "Item"], inplace=True)
df2.drop([201, 202, 40, 41, 42], inplace=True)
df2 = df2.replace({"Iran, Islamic Republic of":"Iran (Islamic Republic of)","Kingdom of Eswatini":"Eswatini", "Republic of North Macedonia":"North Macedonia", "Sint Maarten (Dutch part)":"Sint Maarten", "United Kingdom of Great Britain and Northern Ireland":"The United Kingdom", "Kosovo":"Kosovo[1]","United States":"United States of America","State of Palestine":"occupied Palestinian territory, including east Jerusalem"})
df2 = df2.replace({"Côte d'Ivoire":"Côte d’Ivoire"})
dfT = pd.DataFrame([['United Republic of Tanzania', 81122.12]], columns=["Area", "Value"])
dfT2 = pd.DataFrame([['China', 10261.68]], columns=["Area", "Value"])
df2 = df2.append(dfT, ignore_index=True)
df2 = df2.append(dfT2, ignore_index=True)
df.set_index("Area", inplace=True)
df2.set_index("Area", inplace=True)
merged = pd.merge(df, df2, how="inner", left_index=True, right_index=True)
merged["rate"] = merged.apply(lambda x: (x["Cumulative_deaths"]/x["Cumulative_cases"])*100, axis=1)
print(ttest_ind(merged["rate"], merged["Value"]))
print(merged[merged["Value"]<5000]["rate"].mean())
print(merged[merged["Value"]>30000]["rate"].mean())
sortedM = merged.sort_values(by=['rate'])



Ttest_indResult(statistic=-6.082488977861639, pvalue=1.9051403106041447e-08)
2.5691697490380228
1.4078869907720786


In [401]:
merged["rate"].dtype

dtype('float64')

In [406]:
sortedM

,Cumulative_cases,Cumulative_deaths,Value,rate
Area,,,,
Singapore,59858,29,64102.737611,0.048448
Qatar,159967,256,64781.733197,0.160033
United Arab Emirates,368175,1108,43103.323058,0.300944
Thailand,25415,83,7784.739080,0.326579
Bahrain,117234,420,23503.977127,0.358258
Malaysia,280272,1051,11414.293654,0.374993
Sri Lanka,79480,435,3939.684198,0.547307
Kuwait,183322,1039,31999.271891,0.566762
Cuba,43484,291,9295.901636,0.669212


In [408]:
sortedM

,Cumulative_cases,Cumulative_deaths,Value,rate
Area,,,,
Singapore,59832,29,64102.737611,0.048469
Qatar,159053,256,64781.733197,0.160953
United Arab Emirates,361877,1073,43103.323058,0.296510
Thailand,25241,83,7784.739080,0.328830
Bahrain,115705,414,23503.977127,0.357806
...,...,...,...,...
China,101624,4842,10261.680000,4.764623
Ecuador,269860,15444,6183.823825,5.722967
Egypt,176333,10201,3161.324684,5.785077
